In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.2.0


**Note:** tensorflow_datasets (tfds for short) contains many types of datasets. IMDB reviews is one of them. It contains 50,000 movie reviews that are classified as positive or negative. There are 25,000 samples for training and testing each.

In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete1QIBJF/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete1QIBJF/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete1QIBJF/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


**Note:** The labels and reviews are stored as tensors. A tensor can be thought of as a container that has N-dimensional data. Since we need our data to be an array for training, we use numpy to convert the tensors into arrays.

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

# Need numpy arrays instead of straight arrays for training:  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


**Note:** If sentences (reviews) are shorter than max_length, they will be padded. If they are longer than max_length, they will be truncated.

**Note:** Remember to fit the tokenizer on the training sentences only! Not the testing sentences.

**Note:** Since the tokenizer was fitted on the training set, you should expect to see a lot more OOV tokens when you use it for the test set.

In [0]:
# Hyperparameters on top for easy modification.
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


**Note:** Use reverse_word_index to reverse the keys and values in a dictionary.

**Note:** The get() method takes maximum of two parameters:

1. key - key to be searched in the dictionary
2. value (optional) - Value to be returned if the key is not found. The default value is None.

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3])) # Processed sentence output
print(training_sentences[3]) # Original sentence

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


**Note:** The embedding will be a 2D array of size (No. of features) * (Length of vocabulary). We use max_length to set the same encoding length for all words in the vocabulary.

**Note:** Instead of using Flatten(), which turns all the rows of the array into a single line, you can use Global Average Pooling, which will take the average across all words for each feature. This will return a 1 dimensional vector with the number of elements equal to the number of features. This should make the computations for the Dense layers easier and faster.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Only 2 output classes
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 8s 10ms/step - loss: 0.5063 - accuracy: 0.7334 - val_loss: 0.3482 - val_accuracy: 0.8501
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2385 - accuracy: 0.9094 - val_loss: 0.3680 - val_accuracy: 0.8406
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0912 - accuracy: 0.9773 - val_loss: 0.4508 - val_accuracy: 0.8278
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0240 - accuracy: 0.9970 - val_loss: 0.5276 - val_accuracy: 0.8274
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0092 - accuracy: 0.9990 - val_loss: 0.5713 - val_accuracy: 0.8283
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0047 - accuracy: 0.9994 - val_loss: 0.6316 - val_accuracy: 0.8205
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 0.6806 - val_accuracy: 0.8210
Epoc

**Note:** Training accuracy of 1 from epoch 8 onwards shows we are overfitting!

**Note:** The following code is used to create meta.tsv (actual words) and vecs.tsv (vectors of corresponding words). These are used to visualize the word vectors in the embedding projector at https://projector.tensorflow.org.

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

**Note:** The following code just allows you to download the .tsv files if you're running this notebook in Colab.

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]
